In [114]:
import requests
import pandas as pd
import zipfile
import io
from io import StringIO

In [115]:
dic_tabelas = {
    'tabela_ncm': {
        'nome': 'ncm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM.csv'
    },
    'tabela_sh': {
        'nome': 'sh',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_SH.csv'
    },
    'tabela_cuci': {
        'nome': 'ncm_cuci',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CUCI.csv'
    },
    'tabela_isic': {
        'nome': 'ncm_isic',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_ISIC.csv'
    },
    'tabela_isic_cuci': {
        'nome': 'isic_cuci',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/ISIC_CUCI.csv' #apagar
    },
    'tabela_cgce': {
        'nome': 'ncm_cgce',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CGCE.csv'
    },
    'tabela_fator_agregado': {
        'nome': 'ncm_fat_agreg',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_FAT_AGREG.csv'
    },
    'tabela_ppe': {
        'nome': 'ncm_ppe',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPE.csv'
    },
    'tabela_ppi': {
        'nome': 'ncm_ppi',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPI.csv'
    },
    'tabela_unidade': {
        'nome': 'ncm_unidade',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_UNIDADE.csv'
    },
    'tabela_nbm_ncm': {
        'nome': 'nbm_ncm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NBM_NCM.csv'
    },
    'tabela_nbm': {
        'nome': 'nbm',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/NBM.csv'
    },
    'tabela_uf': {
        'nome': 'estados',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/UF.csv'
    },
    'tabela_via': {
        'nome': 'via',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/VIA.csv'
    },
    'tabela_urf': {
        'nome': 'urf',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/URF.csv'
    },
    'tabela_pais': {
        'nome': 'paises',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS.csv'
    },
    'tabela_blocos': {
        'nome': 'blocos',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS_BLOCO.csv'
    },
    'tabela_municipios': {
        'nome': 'municipios',
        'url': 'https://balanca.economia.gov.br/balanca/bd/tabelas/UF_MUN.csv'
    }
}


In [116]:
def import_full(url): 
    try:
        # Requisição para obter o arquivo ZIP
        response = requests.get(url)
        response.raise_for_status()  # Levanta exceção se a resposta não for 200

        # Abrir o conteúdo como ZIP
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Lista de arquivos dentro do ZIP
            print("Arquivos no zip:", z.namelist())

            # Abrir o primeiro arquivo dentro do ZIP
            with z.open(z.namelist()[0]) as csv_file:
                # Ler o CSV para um DataFrame
                df = pd.read_csv(csv_file, sep=';', encoding='latin1') 

        return df
    
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL: {e}")
    except zipfile.BadZipFile as e:
        print(f"Erro ao abrir o arquivo ZIP: {e}")
    except Exception as e:
        print(f"Erro ao processar o arquivo CSV: {e}")

In [ ]:
import pandas as pd
import requests
from io import StringIO

def import_tabelas_auxiliares(dic_tabelas):
    globais = globals()
    for chave, info in dic_tabelas.items():
        url = info['url']
        nome_variavel = info['nome']

        response = requests.get(url)
        response.raise_for_status()

        try:
            df = pd.read_csv(StringIO(response.text), sep=';')
        except:
            df = pd.read_csv(StringIO(response.text), sep=';')
            df = df.split()
        
        # Criar variável global com o nome especificado
        globais[nome_variavel] = df


: 

In [ ]:
fat_export = import_full('https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/EXP_COMPLETA.zip') # exportação
fat_import = import_full('https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/IMP_COMPLETA.zip') # importação
import_tabelas_auxiliares(dic_tabelas)

Arquivos no zip: ['EXP_COMPLETA.csv']


fat_export: 
OK:
- CO_NCM	
- CO_UNID
- CO_URF
- CO_PAIS
- CO_VIA
- QT_ESTAT
- VL_FOB - é um dos tipos de valor usados no comércio internacional.


ncm:
ok:
- CO_SH6
- CO_PPE	
- CO_PPI	
- CO_FAT_AGREG
- CO_CUCI_ITEM
- CO_CGCE_N3	
- CO_ISIC_CLASSE


In [ ]:
ncm_joined = (
    ncm
    .merge(sh, left_on='CO_SH6', right_on='CO_SH6', how='inner')
    .merge(ncm_ppe, left_on='CO_PPE', right_on='CO_PPE', how='inner')
    .merge(ncm_ppi, left_on='CO_PPI', right_on='CO_PPI', how='inner')
    .merge(ncm_fat_agreg, left_on='CO_FAT_AGREG', right_on='CO_FAT_AGREG', how='inner')
    .merge(ncm_cuci, left_on='CO_CUCI_ITEM', right_on='CO_CUCI_ITEM', how='inner')
    .merge(ncm_cgce, left_on='CO_CGCE_N3', right_on='CO_CGCE_N3', how='inner')
    .merge(ncm_isic, left_on='CO_ISIC_CLASSE', right_on='CO_ISIC_CLASSE', how='inner')
)


In [ ]:
fat_export_joined = (
    fat_export
    .merge(ncm_unidade, left_on='CO_UNID', right_on='CO_UNID', how='inner')
    .merge(urf, left_on='CO_URF', right_on='CO_URF', how='inner')
    .merge(paises, left_on='CO_PAIS', right_on='CO_PAIS', how='inner')
    .merge(via, left_on='CO_VIA', right_on='CO_VIA', how='inner')
)


In [ ]:
final_joined = (
    fat_export_joined
    .merge(ncm_joined, left_on='CO_NCM', right_on='CO_NCM', how='inner')
)


In [ ]:
final_joined

In [ ]:
#final_joined = (
#    fat_export_joined
#    .merge(ncm_joined, left_on='CO_NCM', right_on='CO_NCM', how='inner')
#)